In [1]:
# Importamos las librerías que necesitamos
import numpy as np
# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
import re
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
pd.set_option('display.max_rows', None)       # Show all rows
pd.set_option('display.max_columns', None)    # Show all columns
pd.set_option('display.width', 1000)          # Set the width to accommodate more columns
pd.set_option('display.max_colwidth', None)   # Set maximum column width to None (auto adjust)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
url_campeonatos = f'https://www.basketball-reference.com/teams/'

response = requests.get(url_campeonatos)

if str(response) == '<Response [200]>':
    print(response.status_code)
    sopa = BeautifulSoup(response.content, 'html.parser')
else:
    print('Hay un error')

200


In [3]:
tabla = sopa.find("div", id="div_teams_active").find("table")

In [4]:
filas = tabla.find('tbody').find_all('tr')

In [5]:
team_data = [] # lista vacia para ir colocando todo mi diccionario

for i in filas: #itera sobre cada fila
    # Ignorar filas que sean comentarios o separadores
    if i.get('class') == ['thead']:
        continue
    
    # Obtener el nombre de la franquicia desde el primer 'th'
    franquicia = i.find("th").text.strip()
    
    # Obtener los demás valores desde 'td'
    valores = i.find_all("td")
    
        # Asegúrate de que no haya filas vacías
    
    liga = valores[0].text.strip()
    desde = valores[1].text.strip()
    hasta = valores[2].text.strip()
    anios = valores[3].text.strip()
    partidos = valores[4].text.strip()
    victorias = valores[5].text.strip()
    derrotas = valores[6].text.strip()
    porcentage_vic = valores[7].text.strip()
    playoffs = valores[8].text.strip()
    div_titulos = valores[9].text.strip()
    conf_titulos = valores[10].text.strip()
    campeonatos = valores[11].text.strip()
    
    # Agregar los datos a la lista principal
    team_data.append([
        franquicia, liga, desde, hasta, anios, partidos, victorias, 
        derrotas, porcentage_vic, playoffs, div_titulos, conf_titulos, campeonatos
    ])

# Convertir la lista de listas a un DataFrame de pandas
df = pd.DataFrame(team_data, columns=[
    "Franquicia", "Liga", "Desde", "Hasta", "Anios", "G", "W", "L", "W/L%", 
    "Plyfs", "Div", "Conf", "Champ"
])

In [21]:
len(df['Franquicia'].unique())

62

In [8]:
team_codes = {
    'Atlanta Hawks': 'ATL',
    'St. Louis Hawks': 'STL',
    'Milwaukee Hawks': 'MIL_HAWKS',
    'Tri-Cities Blackhawks': 'TCB',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BKN',
    'New Jersey Nets': 'NJN',
    'New York Nets': 'NYN',
    'New Jersey Americans': 'NJA',
    'Charlotte Hornets': 'CHA',
    'Charlotte Bobcats': 'CHA_BOB',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Denver Rockets': 'DEN_ROCK',
    'Detroit Pistons': 'DET',
    'Fort Wayne Pistons': 'FTW',
    'Golden State Warriors': 'GSW',
    'San Francisco Warriors': 'SFW',
    'Philadelphia Warriors': 'PHW',
    'Houston Rockets': 'HOU',
    'San Diego Rockets': 'SDR',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'San Diego Clippers': 'SDC',
    'Buffalo Braves': 'BUF',
    'Los Angeles Lakers': 'LAL',
    'Minneapolis Lakers': 'MPLS',
    'Memphis Grizzlies': 'MEM',
    'Vancouver Grizzlies': 'VAN',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'NO/Ok. City Hornets': 'NOK',
    'New Orleans Hornets': 'NOH',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Seattle SuperSonics': 'SEA',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Syracuse Nationals': 'SYR',
    'Phoenix Suns': 'PHX',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'Kansas City Kings': 'KCK',
    'Kansas City-Omaha Kings': 'KCO',
    'Cincinnati Royals': 'CIN',
    'Rochester Royals': 'ROC',
    'San Antonio Spurs': 'SAS',
    'Texas Chaparrals': 'TEX',
    'Dallas Chaparrals': 'DAL_CHA',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'New Orleans Jazz': 'NOJ',
    'Washington Wizards': 'WAS',
    'Washington Bullets': 'WSB',
    'Capital Bullets': 'CAP',
    'Baltimore Bullets': 'BAL',
    'Chicago Zephyrs': 'CHI_ZEP',
    'Chicago Packers': 'CHI_PAK'
}


In [22]:
len(team_codes)

62

In [10]:
df['team_code'] = df['Franquicia'].map(team_codes)


In [12]:
df.head(1)

,Franquicia,Liga,Desde,Hasta,Anios,G,W,L,W/L%,Plyfs,Div,Conf,Champ,team_code
0,Atlanta Hawks,NBA,1949-50,2024-25,76,5943,2929,3014,.493,49,12,0,1,ATL


In [13]:
columnas = df.columns.tolist()
columnas.insert(1, columnas.pop(columnas.index('team_code')))

df = df[columnas]

In [20]:
df[df['team_code'] == 'LAL']

,Franquicia,team_code,Liga,Desde,Hasta,Anios,G,W,L,W/L%,Plyfs,Div,Conf,Champ
40,Los Angeles Lakers,LAL,NBA/BAA,1948-49,2024-25,77,6010,3554,2456,.591,64,33,19,17
41,Los Angeles Lakers,LAL,NBA,1960-61,2024-25,65,5171,3097,2074,.599,53,29,19,12
